# Transcription and Summarization of Recorded Speech using Whisper and chatGPT 

### Requirements (using a Python 3.8 conda environment)

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install spacy
!pip install pyaudio
#!pip install tiktoken
!pip install nltk
!python -m spacy download en_core_web_sm

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import openai
import pyaudio
import whisper
import time
import wave # should come with Python

# Apply the API key
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx?"

### Transcription of an online recording

The code here shown extends from the example shown in [betterprogramming](https://betterprogramming.pub/building-a-thought-summarization-app-with-whisper-and-gpt-3-90c2d8653faa).

In [4]:
#KMP_DUPLICATE_LIB_OK=TRUE()
CHANNELS = 1
FORMAT = pyaudio.paInt32
RATE = 44100
CHUNK = 1024
# Record for 20 seconds
time_recording = 20

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,channels=CHANNELS,rate=RATE,input=True,frames_per_buffer=CHUNK)
print("Start recording")
frames = []

seconds = 10
for i in range(0, int(RATE / CHUNK * seconds)):
    data = stream.read(CHUNK)
    frames.append(data)
    

print("Recording stopped")
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open("output.wav", "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


write_text_file = True
audio_file_path = "./output.wav"

# Load the whisper model
model = whisper.load_model("base.en")
#result = model.transcribe(audio_file_path)
result = model.transcribe(audio_file_path, fp16=False, language='English')
if write_text_file==True:    
    with open("rec_wav.txt", "w+") as f:
        f.write(result["text"])

print(result["text"])

Start recording
Recording stopped
 Thank you.


### Transcription of an offline recording

In [ ]:
write_text_file = True

audio_file_path = "./Albert Camus Straße 38.m4a"

# Load the whisper model
model = whisper.load_model("base.en")
#result = model.transcribe(audio_file_path)
result = model.transcribe(audio_file_path, fp16=False, language='English')
if write_text_file==True:    
    with open("rec_m4a.txt", "w+") as f:
        f.write(result["text"])

print(result["text"])

### Summarization

In [ ]:
import openai

with open("rec_wav.txt") as f:
    contents = f.readlines()
print(contents[0])

# Define the prompt for GPT-3
text = contents #result["text"]
prompt = (f"summarize this text using bulletpoints: {text}")

# Get a response from GPT-3
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=1024,
    n =1,
    stop=None,
    temperature=0.5
)

# Get the summary from the response
summary = response.choices[0].text

# Print the summary
print(summary)

### Summarization by chunks

In [ ]:
import openai
import spacy
from spacy.lang.en import English
nlp = spacy.load("en_core_web_sm")

fname = "rec_m4a.txt"
#fname = "idea_2.txt"

def text_to_chunks(text):
    chunks = [[]]
    chunk_total_words = 0

    sentences = nlp(text)

    for sentence in sentences.sents:
        chunk_total_words += len(sentence.text.split(" "))

        if chunk_total_words > 500:
            chunks.append([])
            chunk_total_words = len(sentence.text.split(" "))

        chunks[len(chunks)-1].append(sentence.text)

    return chunks

def summarize_text(text):
    #prompt = f"Summarize the following text in 3 sentences:\n{text}"
    prompt = (f"summarize this text: {text}")

    response = openai.Completion.create(
      engine="text-davinci-003", 
      prompt=prompt,
      temperature=0.5, 
      max_tokens=1049, 
      top_p=1, 
      frequency_penalty=0,
      presence_penalty=1
    )

    return response["choices"][0]["text"]


with open(fname) as f:
    contents = f.readlines()
print(contents[0])

text = contents[0] #result["text"]

chunks = text_to_chunks(text)

chunk_summaries = []

for idx, chunk in enumerate(chunks):
    chunk_summary = summarize_text(" ".join(chunk))
    print("Summary of chunk:", idx+1)
    chunk_summaries.append(chunk_summary)

summary = " ".join(chunk_summaries)
print("Append summaries")

# Print the summary
print("Summary : ", summary)


The Chilean Civil War of 1891 (also known as Revolution of 1891) was a civil war in Chile fought between forces supporting Congress and forces supporting the President, José Manuel Balmaceda from 16 January 1891 to 18 September 1891. The war saw a confrontation between the Chilean Army and the Chilean Navy, siding with the president and the congress, respectively. This conflict ended with the defeat of the Chilean Army and the presidential forces, and with President Balmaceda committing suicide as a consequence of the defeat.[2] In Chilean historiography the war marks the end of the Liberal Republic and the beginning of the Parliamentary Era.The Chilean Civil War grew out of political disagreements between the president of Chile, José Manuel Balmaceda, and the Chilean congress. In 1889, the congress became distinctly hostile to the administration of Balmaceda, and the political situation became serious, at times threatening to involve the country in civil war. According to usage and cu